In [14]:
import os
import yaml
import chatgpt
import perplexity

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PPLX_API_KEY = os.environ["PPLX_API_KEY"]

# Step 1. 기업 맞춤형 제안서 작성을 위해 타겟 기업 관련 정보 추출
 
# 기업 맞춤형 메일을 작성해야하므로 타겟 기업에 대한 정보를 글로 긁어와야함. 
# 단순히 투자유치 받았다 이런 기사는 쓸모없고, 기업이 어떤 문제를 어떤 접근방식으로 풀고있는지에 대한 정보가 중요함.

# [접근 방법]
# https://www.perplexity.ai api로 검색엔진 돌려서 요약해준 기업 관련 정보 사용

################################################################################################
# --> TODO: INPUT.yaml에 기업명(필수)와 기업 홈페이지 링크(선택)을 yaml 형식에 맞추어 적어주세요    #
################################################################################################

# 타겟 기업 홈페이지 링크 첨부할 시 추출된 정보의 질이 매우 높아진다. 링크를 찾을 수 없거나 홈페이지가 운영을 멈췄으면 따로 홈페이지 링크를 적지 않아도 무방.

# 정보 추출 플로우는 다음 2가지 질문에 대한 생성 결과를 concatenate하는 방식이다.
# (1): '기업과 관련된 정보를 비즈니스 모델을 포함하여 조사해줘'
# (2): '기업에서 해결 중일 것으로 추정되는 문제를 최대한 다양하게 근거와 함께 제시해줘'
#  --> 추출된 기업 정보는 sources/SOURCE_target_company_information.txt에 저장되어 확인 가능하다.


with open('INPUT.yaml', encoding='UTF8') as f:
    input_info = yaml.full_load(f)
    
    
if input_info['company'] is None:
    raise Exception("INPUT.yaml 파일에서 기업명 작성해주세요.")     


prompt_company_info_sys = "해당 기업과 관련된 정보 찾아줘. 반드시 검색엔진에 정보가 있으니 관련된 모든 정보 찾아내."


if input_info['homepage_link'] is None: # INPUT.yaml 에 타겟 기업 홈페이지 링크를 기입하지 않았을 때의 프롬프트
    prompt_business_model_usr = f"""
    
    {input_info['company']} 기업에서 제공하는 서비스 관련 정보를 비즈니스 모델 위주로 정리해줘.
    
    """
    prompt_solving_problem_usr = f"""
    
    {input_info['company']} 기업에서 해결 중일 것으로 추정되는 문제를 최대한 다양하게 근거와 함께 제시해줘.
    
    """
    
    
else: # INPUT.yaml 에 타겟 기업 홈페이지 링크를 기입했을 때의 프롬프트
    prompt_business_model_usr = f"""
    
    {input_info['homepage_link']}
    
    위 링크를 참조하여 {input_info['company']} 기업에서 제공하는 서비스 관련 정보를 비즈니스 모델 위주로 정리해줘.
    만약 해당 링크가 운영 중이 아니라면 검색엔진에 검색하여 기업 정보를 찾아내.
    
    """
    prompt_solving_problem_usr = f"""
    
    {input_info['homepage_link']}
    
    위 링크를 참조하여 {input_info['company']} 기업에서 해결 중일 것으로 추정되는 문제를 최대한 다양하게 근거와 함께 제시해줘.
    만약 해당 링크가 운영 중이 아니라면 검색엔진에 검색하여 기업 정보를 찾아내.
    
    """
    
# 1) 비즈니스 모델 조사
prompt = [
    {"role":"system", "content":prompt_company_info_sys},
    {"role":"user", "content":prompt_business_model_usr}
]

business_model_info = perplexity.run_chat_completion(PPLX_API_KEY, prompt).choices[0].message.content

# 2) 현재 풀고 있는 문제 조사
prompt = [
    {"role":"system", "content":prompt_company_info_sys},
    {"role":"user", "content":prompt_solving_problem_usr}
]

solving_problem_info = perplexity.run_chat_completion(PPLX_API_KEY, prompt).choices[0].message.content




f= open("sources/SOURCE_target_company_information.txt","w+", encoding='UTF8')

target_company_information = business_model_info + "\n\n" + solving_problem_info
f.write(target_company_information)
f.close()

#################################################################################################################################################3


# 보너스) 담당자 영문명을 표준 로마자 표기법에 의거해 한국어명으로 변환 
# 그저 편의를 위한 기능(-> 이름 변환하는 과정에서 잘못된 변환이 잦으면 이 부분을 삭제하고 수동 방식으로 바꿔도 무방합니다.) 

personal_info_sys = open("prompts/PROMPT_personal_info_transform.txt", 'r', encoding='UTF8').read()
personal_info_usr = f"""

[담당자 정보]
성함: {input_info['name']}
직함: {input_info['position']}

"""
prompt = [{"role":"system", "content":personal_info_sys},
            {"role":"user", "content":personal_info_usr}
        ]
transformed_personal_info = chatgpt.run_chat_completion(OPENAI_API_KEY, prompt).choices[0].message.content


################################################################################################################################3

# Step 2. 타겟 기업과 관련된 과거 프로젝트 선별하기

# 기업 맞춤형 제안서 작성을 위해 과거에 진행했던 프로젝트 중 타겟 기업의 도메인과 유사한 도메인이면서 비슷한 문제를 다룬 프로젝트를 3개 선별합니다.
# 이 단계에서 선별된 프로젝트는 '유관 프로젝트'로서 이후 제안서 템플릿 특정 구역에 그대로 삽입됩니다.


summarized_projects = open("sources/SOURCE_summarized_project.txt", 'r', encoding='UTF8').read()


# 만약 타겟 기업 정보 조사를 perplexity를 이용하지 않고 직접 하고싶다면 sources/SOURCE_target_company_information.txt에 텍스트로 저장하고 다음과 같이 불러옵니다.
# target_company_information = open("sources/SOURCE_target_company_information.txt", 'r', encoding='UTF8').read()

select_relative_proj_sys = open("prompts/PROMPT_select_relative_project.txt", 'r', encoding='UTF8').read()
select_relative_proj_usr = """
다음 내용을 바탕으로 타겟 기업과 관련된 프로젝트를 뽑아서 출력해줘


[타겟 기업 관련 정보]

{target_company_information}

[지난 여러 기업과 진행한 프로젝트들]

{summarized_projects}

"""

prompt = [
            {"role":"system", "content":select_relative_proj_sys},
            {"role":"user", "content":select_relative_proj_usr.format(target_company_information = target_company_information, summarized_projects = summarized_projects)}
        ]

selected_relative_project = chatgpt.run_chat_completion(OPENAI_API_KEY, prompt).choices[0].message.content




############################################################################################################

# Step 3. 제안서 템플릿의 빈부분 채우기

# 조사된 기업 정보를 바탕으로 제안서 템플릿을 적절하게 채웁니다.
# 우리가 타겟 기업에 대해 잘 알고있다는 것을 어필하는 것이 목적입니다.
# 이 파트의 프롬프트는 굉장히 길고 구체적입니다. prompts/PROMPT_complete_proposal_letter.txt에서 확인할 수 있습니다.


template = open("sources/SOURCE_proposal_template.txt", 'r', encoding='UTF8').read()
template = template.format(selected_relative_project=selected_relative_project)

prompt_sys = open("prompts/PROMPT_complete_proposal_letter.txt", 'r', encoding='UTF8').read()
prompt_usr = """

다음 내용을 바탕으로 완전한 협업 제안 메일을 발송해줘.

[담당자 정보]

{transformed_personal_info}



[타겟 기업 관련 정보]

{target_company_information}



[관련 프로젝트]

{selected_relative_project}



[제안서 템플릿]

{template}



"""

# 작성된 맞춤형 제안서는 outputs 폴더에 기업명(항상), 담당자명(담당자명을 기입했을 시)과 함께 txt 파일로 누적 저장됩니다.
f= open(f"outputs/{input_info['company']}{("_"+input_info['name']) if input_info['name'] is not None else ""}_completed_proposal_letter.txt","w+", encoding='UTF8')
    
prompt = [{"role":"system", "content":prompt_sys},
            {"role":"user", "content":prompt_usr.format(target_company_information = target_company_information, template = template, transformed_personal_info=transformed_personal_info, selected_relative_project=selected_relative_project)}
        ]
answer = chatgpt.run_chat_completion(OPENAI_API_KEY, prompt)
f.write(answer.choices[0].message.content)

f.close()